In [3]:
import pandas as pd

In [4]:
df_lag = pd.read_parquet("sell_in_norm.parquet")

In [5]:
df_lag['periodo_dt'] = pd.to_datetime(df_lag['periodo'], format='%Y%m')
df_lag = df_lag.sort_values(by=['product_id', 'periodo_dt'])

for lag in range(1, 13):
    df_lag[f'tn_lag_{lag}'] = df_lag.groupby('product_id')['tn_norm'].shift(lag)

df_lag = df_lag.sort_values(by=['product_id', 'periodo'])

In [6]:
df_lag.to_parquet('sell_in_lag.parquet', index=False)